### Puzzle 1

In [7]:
n_increases = 0
prev_x = 9999999999999
with open("advent1.txt", 'r') as fin:
    for line in fin.readlines():
        x = int(line)
        if x > prev_x:
            n_increases += 1
        prev_x = x
n_increases

1466

### Puzzle 2

In [16]:
n_increases = 0
window = []
prev_sum = None
with open("advent1.txt", 'r') as fin:
    for line in fin.readlines():
        x = int(line)
        if len(window) < 3:
            window.append(x)
        if prev_sum is None:
            prev_sum = sum(window)
        if len(window) == 3:
            window.pop(0)
            window.append(x)
            new_sum = sum(window)
            if new_sum > prev_sum:
                n_increases += 1
            prev_sum = new_sum
n_increases

1491